### From Github

In [ ]:
!git clone https://github.com/AshwinRJ/Federated-Learning-PyTorch.git

Cloning into 'Federated-Learning-PyTorch'...
remote: Enumerating objects: 261, done.
remote: Total 261 (delta 0), reused 0 (delta 0), pack-reused 261
Receiving objects: 100% (261/261), 214.52 KiB | 4.88 MiB/s, done.
Resolving deltas: 100% (136/136), done.


In [ ]:
!python '/content/Federated-Learning-PyTorch/src/baseline_main.py' --model=mlp --dataset=mnist --epochs=2

In [ ]:
pip install tensorboardX

In [ ]:
!python '/content/Federated-Learning-PyTorch/src/federated_main.py' --model=cnn --dataset=cifar --gpu=0 --iid=1 --epochs=2

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
import math

a1 = 0.0
a2 = 0.0

H = 1.0
M = 0.5
L = 0.0

H1 = 0.0
M1 = 0.0
L1 = 0.0

H2 = 0.0
M2 = 0.0
L2 = 0.0

numberOfAntAttributes = 2
matchingDegree = [0.0] * 9
relativeWeight = 1.0
totalWeight = 0.0
consequentBeliefDegree = [0.0] * 27
updatedConsequentBeliefDegree = [0.0] * 27
beliefDegreeChangeLevel = 0.0
activationWeight = [0.0] * 9
ruleWiseBeliefDegreeSum = [0.0] * 9

def transformInput1(i):
    global H1, M1, L1
    if i >= H:
        H1 = 1.0
        M1 = 0.0
        L1 = 0.0
    elif i == M:
        H1 = 0.0
        M1 = 1.0
        L1 = 0.0
    elif i <= L:
        H1 = 0.0
        M1 = 0.0
        L1 = 1.0
    elif (i <= H) and (i >= M):
        M1 = (H - i) / (H - M)
        H1 = 1.0 - M1
        L1 = 0.0
    elif (i <= M) and (i >= L):
        L1 = (M - i) / (M - L)
        M1 = 1.0 - L1
        H1 = 0.0

def transformInput2(i):
    global H2, M2, L2
    if i >= H:
        H2 = 1.0
        M2 = 0.0
        L2 = 0.0
    elif i == M:
        H2 = 0.0
        M2 = 1.0
        L2 = 0.0
    elif i <= L:
        H2 = 0.0
        M2 = 0.0
        L2 = 1.0
    elif (i <= H) and (i >= M):
        M2 = (H - i) / (H - M)
        H2 = 1.0 - M2
        L2 = 0.0
    elif (i <= M) and (i >= L):
        L2 = (M - i) / (M - L)
        M2 = 1.0 - L2
        H2 = 0.0

def ruleBase():
    global consequentBeliefDegree
    consequentBeliefDegree[0] = 1.0
    consequentBeliefDegree[1] = 0.0
    consequentBeliefDegree[2] = 0.0
    consequentBeliefDegree[3] = 0.0
    consequentBeliefDegree[4] = 0.40
    consequentBeliefDegree[5] = 0.60
    consequentBeliefDegree[6] = 0.0
    consequentBeliefDegree[7] = 0.0
    consequentBeliefDegree[8] = 1.0
    consequentBeliefDegree[9] = 1.0
    consequentBeliefDegree[10] = 0.0
    consequentBeliefDegree[11] = 0.0
    consequentBeliefDegree[12] = 0.0
    consequentBeliefDegree[13] = 1.0
    consequentBeliefDegree[14] = 0.0
    consequentBeliefDegree[15] = 0.0
    consequentBeliefDegree[16] = 0.40
    consequentBeliefDegree[17] = 0.60
    consequentBeliefDegree[18] = 0.0
    consequentBeliefDegree[19] = 0.0
    consequentBeliefDegree[20] = 1.0
    consequentBeliefDegree[21] = 0.60
    consequentBeliefDegree[22] = 0.40
    consequentBeliefDegree[23] = 0.0
    consequentBeliefDegree[24] = 0.0
    consequentBeliefDegree[25] = 0.0
    consequentBeliefDegree[26] = 1.0

def takeInput():
    global a1, a2
    print("Insert value for A1 (between 0 and 1): ")
    a1 = float(input())
    print("Insert value for A2 (between 0 and 1): ")
    a2 = float(input())
    transformInput1(a1)
    transformInput2(a2)


def show_transformed_input():
    print(f"\nTransformed Input is as follow.")
    print(f"A1 = {{(H, {H1}); (M, {M1}); (L, {L1})}}")
    print(f"A2 = {{(H, {H2}); (M, {M2}); (L, {L2})}}")

def calculate_matching_degree():
    increment = 0
    ti1 = [H1, M1, L1]
    ti2 = [H2, M2, L2]

    for c in range(3):
        for d in range(3):
            matchingDegree[increment] = pow(ti1[c], relativeWeight) * pow(ti2[d], relativeWeight)
            increment += 1

def show_matching_degree():
    print(f"\nMatching degrees of the rules are as follow.")
    for counter in range(9):
        print(f"Matching Degree of Rule {counter+1} = {matchingDegree[counter]}")

def show_activation_weight():
    total_weight = 0
    for x in range(9):
        total_weight += matchingDegree[x]

    print(f"\nActivation Weights of the rules are as follow.")
    for counter in range(9):
        activationWeight[counter] = matchingDegree[counter] / total_weight
        print(f"Activation weight of Rule {counter+1} = {activationWeight[counter]}")


def update_belief_degree():
    update = 0
    sum_ant_attr1 = 1
    sum_ant_attr2 = 1

    if (H1 + M1 + L1) < 1:
        sum_ant_attr1 = H1 + M1 + L1
        update = 1

    if (H2 + M2 + L2) < 1:
        sum_ant_attr2 = H2 + M2 + L2
        update = 1

    if update == 1:
        belief_degree_change_level = (sum_ant_attr1 + sum_ant_attr2) / numberOfAntAttributes
        print(f"Belief Degree Change Level = {belief_degree_change_level}")
        for go in range(27):
            consequentBeliefDegree[go] = belief_degree_change_level * consequentBeliefDegree[go]
            print(f"Updated Consequent Belief Degree : {consequentBeliefDegree[go]}")
    else:
        print("\nNo upgradation of belief degree required.")


def aggregateER():
    parse = 0
    move1 = 0
    move2 = 1
    move3 = 2
    action1 = 0
    action2 = 1
    action3 = 2

    part11 = 1
    part12 = 1
    part13 = 1
    flpart1 = 1
    part2 = 1.0
    value = 1.0
    meu = 1.0

    numeratorH1 = 1.0
    numeratorH2 = 1.0
    numeratorH = 1.0
    denominatorH1 = 1.0
    denominatorH = 1.0
    aggregatedBeliefDegreeH = 1.0
    numeratorM1 = 1.0
    numeratorM = 1.0
    aggregatedBeliefDegreeM = 1.0
    numeratorL1 = 1.0
    numeratorL = 1.0
    aggregatedBeliefDegreeL = 1.0
    utilityScoreH = 1.0
    utilityScoreM = 0.5
    utilityScoreL = 0.0
    crispValue = 1.0
    degreeOfIncompleteness = 1.0
    utilityMax = 1.0
    utilityMin = 1.0
    utilityAvg = 1.0

    for t in range(9):
        parse = t * 3
        ruleWiseBeliefDegreeSum[t] = (consequentBeliefDegree[parse] +consequentBeliefDegree[parse + 1] + consequentBeliefDegree[parse + 2])


    for rule in range(9):
        part11 *= (activationWeight[rule] * consequentBeliefDegree[move1] + 1 - (activationWeight[rule] * ruleWiseBeliefDegreeSum[rule]))
        move1 += 3


    for rule in range(9):
        part12 *= (activationWeight[rule] * consequentBeliefDegree[move2] + 1 - (activationWeight[rule] * ruleWiseBeliefDegreeSum[rule]))
        move2 += 3


    for rule in range(9):
        part13 *= (activationWeight[rule] * consequentBeliefDegree[move3] + 1 - (activationWeight[rule] * ruleWiseBeliefDegreeSum[rule]))
        move3 += 3


    part1 = (part11 + part12 + part13)


    for rule in range(9):
        part2 *= (1 - (activationWeight[rule] * ruleWiseBeliefDegreeSum[rule]))


    value = part1 - part2

    meu = 1/value


    for rule in range(9):
        numeratorH1 *= (activationWeight[rule] * consequentBeliefDegree[action1] + 1 - (activationWeight[rule] * ruleWiseBeliefDegreeSum[rule]))
        action1 += 3


    for rule in range(9):
        numeratorH2 *= (1 - (activationWeight[rule] * ruleWiseBeliefDegreeSum[rule]))


    numeratorH = meu * (numeratorH1 - numeratorH2)


    for rule in range(9):
        denominatorH1 *= (1 - activationWeight[rule])

    denominatorH = 1 - (meu * denominatorH1)
    aggregatedBeliefDegreeH = numeratorH / denominatorH
    print("ER Aggregated Belief Degree for High:", aggregatedBeliefDegreeH)

    for rule in range(9):
        numeratorM1 *= (activationWeight[rule] * consequentBeliefDegree[action2] + 1 - (activationWeight[rule] * ruleWiseBeliefDegreeSum[rule]))
        action2 += 3


    numeratorM = meu * (numeratorM1 - numeratorH2)
    aggregatedBeliefDegreeM = (numeratorM/denominatorH)
    print(f"ER Aggregated Belief Degree for Medium: {aggregatedBeliefDegreeM}")

    for rule in range(9):
        numeratorL1 *= (activationWeight[rule] * consequentBeliefDegree[action3] + 1 - (activationWeight[rule] * ruleWiseBeliefDegreeSum[rule]))
        action3 += 3

    numeratorL = meu * (numeratorL1 - numeratorH2)
    aggregatedBeliefDegreeL = numeratorL / denominatorH
    print("ER Aggregated Belief Degree for Low:", aggregatedBeliefDegreeL)

    if (aggregatedBeliefDegreeH + aggregatedBeliefDegreeM + aggregatedBeliefDegreeL) == 1:
        crispValue = (aggregatedBeliefDegreeH * utilityScoreH) + (aggregatedBeliefDegreeM * utilityScoreM) + (aggregatedBeliefDegreeL * utilityScoreL)
        print("Crisp or numerical value is:", crispValue)

    else:
        degreeOfIncompleteness = 1 - (aggregatedBeliefDegreeH + aggregatedBeliefDegreeM + aggregatedBeliefDegreeL)
        print("Unassigned Degree of Belief:", degreeOfIncompleteness)

    utilityMax = ((aggregatedBeliefDegreeH + degreeOfIncompleteness) * utilityScoreH + (aggregatedBeliefDegreeM * utilityScoreM) + (aggregatedBeliefDegreeL * utilityScoreL))

    utilityMin = (aggregatedBeliefDegreeH * utilityScoreH) + (aggregatedBeliefDegreeM * utilityScoreM) + (aggregatedBeliefDegreeL + degreeOfIncompleteness) * utilityScoreL

    utilityAvg = (utilityMax + utilityMin) / 2

    print("Maximum expected utility:", utilityMax)
    print("Minimum expected utility:", utilityMin)
    print("Average expected utility:", utilityAvg)

    print("Revised ER Aggregated Belief Degrees considering degree of Incompleteness are as follows:")
    print("Revised ER Aggregated Belief Degree for High:", aggregatedBeliefDegreeH / (aggregatedBeliefDegreeH + aggregatedBeliefDegreeM + aggregatedBeliefDegreeL))
    print("Revised ER Aggregated Belief Degree for Medium:", aggregatedBeliefDegreeM / (aggregatedBeliefDegreeH + aggregatedBeliefDegreeM + aggregatedBeliefDegreeL))
    print("Revised ER Aggregated Belief Degree for Low:", aggregatedBeliefDegreeL / (aggregatedBeliefDegreeH + aggregatedBeliefDegreeM + aggregatedBeliefDegreeL))

def main():
    ruleBase()
    takeInput()
    show_transformed_input()
    calculate_matching_degree()
    show_matching_degree()
    show_activation_weight()
    update_belief_degree()
    aggregateER()

if __name__ == "__main__":
    main()


Insert value for A1 (between 0 and 1): 
0.8
Insert value for A2 (between 0 and 1): 
0.5

Transformed Input is as follow.
A1 = {(H, 0.6000000000000001); (M, 0.3999999999999999); (L, 0.0)}
A2 = {(H, 0.0); (M, 1.0); (L, 0.0)}

Matching degrees of the rules are as follow.
Matching Degree of Rule 1 = 0.0
Matching Degree of Rule 2 = 0.6000000000000001
Matching Degree of Rule 3 = 0.0
Matching Degree of Rule 4 = 0.0
Matching Degree of Rule 5 = 0.3999999999999999
Matching Degree of Rule 6 = 0.0
Matching Degree of Rule 7 = 0.0
Matching Degree of Rule 8 = 0.0
Matching Degree of Rule 9 = 0.0

Activation Weights of the rules are as follow.
Activation weight of Rule 1 = 0.0
Activation weight of Rule 2 = 0.6000000000000001
Activation weight of Rule 3 = 0.0
Activation weight of Rule 4 = 0.0
Activation weight of Rule 5 = 0.3999999999999999
Activation weight of Rule 6 = 0.0
Activation weight of Rule 7 = 0.0
Activation weight of Rule 8 = 0.0
Activation weight of Rule 9 = 0.0

No upgradation of belief deg

### Fedarated Learning

In [ ]:
!python '/content/drive/MyDrive/FLCode/mainBRB.py'

Insert value for A1 (between 0 and 1): 

^C


In [ ]:
# importing the zipfile module
from zipfile import ZipFile

# loading the temp.zip and creating a zip object
with ZipFile("/content/drive/MyDrive/FLCode/archive.zip", 'r') as zObject:

	# Extracting all the members of the zip
	# into a specific location.
	zObject.extractall(path="MNISTDataset/")


In [ ]:
!git clone 'https://github.com/stijani/tutorial.git'

Cloning into 'tutorial'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 7 (delta 0), reused 2 (delta 0), pack-reused 0
Receiving objects: 100% (7/7), 4.92 KiB | 4.92 MiB/s, done.


In [ ]:
import numpy as np
import random
import cv2
import os
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K

from fl_mnist_implementation_tutorial_utils import *
from mainBRB import *

In [ ]:
def load(paths, verbose=-1):
    '''expects images for each class in seperate dir,
    e.g all digits in 0 class in the directory named 0 '''
    data = list()
    labels = list()
    # loop over the input images
    for (i, imgpath) in enumerate(paths):
        # load the image and extract the class labels
        im_gray = cv2.imread(imgpath, cv2.IMREAD_GRAYSCALE)
        image = np.array(im_gray).flatten()
        label = imgpath.split(os.path.sep)[-2]
        # scale the image to [0, 1] and add to list
        data.append(image/255)
        labels.append(label)
        # show an update every `verbose` images
        if verbose > 0 and i > 0 and (i + 1) % verbose == 0:
            print("[INFO] processed {}/{}".format(i + 1, len(paths)))
    # return a tuple of the data and labels
    return data, labels

In [ ]:
#declear path to your mnist data folder
img_path = '/content/MNISTDataset/trainingSet/trainingSet'

#get the path list using the path object
image_paths = list(paths.list_images(img_path))

#apply our function
image_list, label_list = load(image_paths, verbose=10000)

#binarize the labels
lb = LabelBinarizer()
label_list = lb.fit_transform(label_list)

#split data into training and test set
X_train, X_test, y_train, y_test = train_test_split(image_list,
                                                    label_list,
                                                    test_size=0.1,
                                                    random_state=42)

[INFO] processed 10000/42000
[INFO] processed 20000/42000
[INFO] processed 30000/42000
[INFO] processed 40000/42000


In [ ]:
def create_clients(image_list, label_list, num_clients=10, initial='clients'):
    ''' return: a dictionary with keys clients' names and value as
                data shards - tuple of images and label lists.
        args:
            image_list: a list of numpy arrays of training images
            label_list:a list of binarized labels for each image
            num_client: number of fedrated members (clients)
            initials: the clients'name prefix, e.g, clients_1

    '''

    #create a list of client names
    client_names = ['{}_{}'.format(initial, i+1) for i in range(num_clients)]

    #randomize the data
    data = list(zip(image_list, label_list))
    random.shuffle(data)

    #shard data and place at each client
    size = len(data)//num_clients
    shards = [data[i:i + size] for i in range(0, size*num_clients, size)]

    #number of clients must equal number of shards
    assert(len(shards) == len(client_names))

    return {client_names[i] : shards[i] for i in range(len(client_names))}

In [ ]:
#create clients
clients = create_clients(X_train, y_train, num_clients=10, initial='client')

In [ ]:
def batch_data(data_shard, bs=32):
    '''Takes in a clients data shard and create a tfds object off it
    args:
        shard: a data, label constituting a client's data shard
        bs:batch size
    return:
        tfds object'''
    #seperate shard into data and labels lists
    data, label = zip(*data_shard)
    dataset = tf.data.Dataset.from_tensor_slices((list(data), list(label)))
    return dataset.shuffle(len(label)).batch(bs)

In [ ]:
#process and batch the training data for each client
clients_batched = dict()
for (client_name, data) in clients.items():
    clients_batched[client_name] = batch_data(data)
    print(client_name)
    print(list(clients_batched[client_name]))
    print(list(batch_data(data)))
#process and batch the test set
test_batched = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(len(y_test))

In [ ]:
class SimpleMLP:
    @staticmethod
    def build(shape, classes):
        model = Sequential()
        model.add(Dense(200, input_shape=(shape,)))
        model.add(Activation("relu"))
        model.add(Dense(200))
        model.add(Activation("relu"))
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        return model

In [ ]:
learning_rate = 0.01
comms_round = 100
#decay=learning_rate/comms_round
loss='categorical_crossentropy'
metrics = ['accuracy']
optimizer = tf.keras.optimizers.legacy.SGD(learning_rate=learning_rate,
                decay=learning_rate/comms_round,
                momentum=0.9
               )

In [ ]:
def weight_scalling_factor(clients_trn_data, client_name):
    client_names = list(clients_trn_data.keys())
    #get the bs
    bs = list(clients_trn_data[client_name])[0][0].shape[0]
    #first calculate the total training data points across clinets
    global_count = sum([tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy() for client_name in client_names])*bs
    # get the total number of data points held by a client
    local_count = tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy()*bs
    return local_count/global_count


def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final



def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)

    return avg_grad


def test_model(X_test, Y_test,  model, comm_round):
    cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    #logits = model.predict(X_test, batch_size=100)
    logits = model.predict(X_test)
    loss = cce(Y_test, logits)
    acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(Y_test, axis=1))
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    return acc, loss

In [ ]:
#initialize global model
smlp_global = SimpleMLP()
global_model = smlp_global.build(784, 10)

#commence global training loop
for comm_round in range(comms_round):

    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()

    #initial list to collect local model weights after scalling
    scaled_local_weight_list = list()

    #randomize client data - using keys
    client_names= list(clients_batched.keys())
    random.shuffle(client_names)

    #loop through each client and create new local model
    '''for client in client_names:
        smlp_local = SimpleMLP()
        local_model = smlp_local.build(784, 10)
        local_model.compile(loss=loss,
                      optimizer=optimizer,
                      metrics=metrics)

        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)

        #fit local model with client's data
        local_model.fit(clients_batched[client], epochs=1, verbose=0)

        #scale the model weights and add to list
        scaling_factor = weight_scalling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)

        #clear session to free memory after each communication round
        K.clear_session()'''
    # Integration of BRB
    for client in client_names:
        smlp_local = SimpleMLP()
        local_model = smlp_local.build(784, 10)
        local_model.compile(loss=loss,
                      optimizer=optimizer,
                      metrics=metrics)

        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)

        #updated_attribute_weights = calculate_updated_attribute_weights(clients_batched[client])  # Assuming client data is used for BRB calculation
        #apply_updated_attribute_weights(local_model, updated_attribute_weights)  # Apply updated attribute weights to the local model
        print(f'client: {client}')
        print(f'client_batched: {clients_batched[client]}')
        #fit local model with client's data
        local_model.fit(clients_batched[client], epochs=1, verbose=0)

        #scale the model weights and add to list
        scaling_factor = weight_scalling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)

        #clear session to free memory after each communication round
        K.clear_session()


    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights = sum_scaled_weights(scaled_local_weight_list)

    #update global model
    global_model.set_weights(average_weights)

    #test global model and print out metrics after each communications round
    for(X_test, Y_test) in test_batched:
        global_acc, global_loss = test_model(X_test, Y_test, global_model, comm_round)

NameError: name 'SimpleMLP' is not defined

In [ ]:
def test_model(X_test, Y_test,  model, comm_round):
    cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    #logits = model.predict(X_test, batch_size=100)
    logits = model.predict(X_test)
    loss = cce(Y_test, logits)
    acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(Y_test, axis=1))
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    return acc, loss

In [ ]:
SGD_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(len(y_train)).batch(320)
smlp_SGD = SimpleMLP()
SGD_model = smlp_SGD.build(784, 10)

SGD_model.compile(loss=loss,
              optimizer=optimizer,
              metrics=metrics)

# fit the SGD training data to model
_ = SGD_model.fit(SGD_dataset, epochs=100, verbose=0)

#test the SGD global model and print out metrics
for(X_test, Y_test) in test_batched:
        SGD_acc, SGD_loss = test_model(X_test, Y_test, SGD_model, 1)

In [ ]:
def non_iid_x(image_list, label_list, x=1, num_intraclass_clients=10):
        ''' creates x non_IID clients
        args:
            image_list: python list of images or data points
            label_list: python list of labels
            x: none IID severity, 1 means each client will only have one class of data
            num_intraclass_client: number of sub-client to be created from each none IID class,
            e.g for x=1, we could create 10 further clients by splitting each class into 10

        return - dictionary
            keys - clients's name,
            value - client's non iid 1 data shard (as tuple list of images and labels) '''

        non_iid_x_clients = dict()

        #create unique label list and shuffle
        unique_labels = np.unique(np.array(label_list))
        random.shuffle(unique_labels)

        #create sub label lists based on x
        sub_lab_list = [unique_labels[i:i + x] for i in range(0, len(unique_labels), x)]

        for item in sub_lab_list:
            class_data = [(image, label) for (image, label) in zip(image_list, label_list) if label in item]

            #decouple tuple list into seperate image and label lists
            images, labels = zip(*class_data)

            # create formated client initials
            initial = ''
            for lab in item:
                initial = initial + lab + '_'

            #create num_intraclass_clients clients from the class
            intraclass_clients = create_clients(list(images), list(labels), num_intraclass_clients, initial)

            #append intraclass clients to main clients'dict
            non_iid_x_clients.update(intraclass_clients)

        return non_iid_x_clients